<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#takes 50mins to load
!TORCH_CUDA_ARCH_LIST="7.5+PTX" pip install git+https://github.com/casper-hansen/AutoAWQ_kernels.git@main

In [ ]:
!pip install git+https://github.com/casper-hansen/AutoAWQ.git@main

In [ ]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
!pip install openai==0.28


In [2]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [3]:
import os
os.environ["APIFY_API_TOKEN"] = "apify_api_r2J5EYAZ2qOeCTSBDqN3RJ2IraTke911bWjC"
os.environ["OPENAI_API_KEY"] = ""


In [8]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest just one articles
latest_articles = feed.entries[4:5]

In [9]:
apify = ApifyWrapper()

In [10]:
urls_to_crawl = []
# Loop through the latest articles
for article in latest_articles:
    # Get the title and article URL
    title = article.title
    article_url = article.link
    # Print the article URL
    #print(article_url)
    # Append the article URL to the list of URLs to crawl
    urls_to_crawl.append({"url": article_url})

# Use the list of URLs to call the Apify actor
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [11]:
t = loader.load()
t

[Document(page_content='NNPC Exploration and Production Limited (NNPC E&P Ltd.), NNPC Ltd.’s flagship upstream subsidiary, and Natural Oilfield Services Ltd. (NOSL), a subsidiary of Sterling Oil Exploration & Energy Production Company Ltd. (SEEPCO), have announced the successful commencement of oil production at Oil Mining Lease (OML) 13 in Akwa Ibom State, Nigeria.\nThis was revealed by the Chief Corporate Communications Officer, NNPC Ltd, Olufemi Soneye.\nIt reads,” The production, which commenced on the 6th of May 2024, with 6,000 barrels of oil, is expected to be ramped up to 40,000 barrels per day by May 27th, 2024.\n“The first oil flow from OML 13 is a historic milestone in the partnership between NNPC E&P Ltd and NOSL, highlighting their dedication to driving growth and development in Nigeria’s oil and gas sector, which remains a vital component of the nation’s economy.\n“The achievement does not only signify the culmination of rigorous planning and execution by the teams involv

In [12]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in t]

# Printing the extracted content
print(extracted_content)

['NNPC Exploration and Production Limited (NNPC E&P Ltd.), NNPC Ltd.’s flagship upstream subsidiary, and Natural Oilfield Services Ltd. (NOSL), a subsidiary of Sterling Oil Exploration & Energy Production Company Ltd. (SEEPCO), have announced the successful commencement of oil production at Oil Mining Lease (OML) 13 in Akwa Ibom State, Nigeria.\nThis was revealed by the Chief Corporate Communications Officer, NNPC Ltd, Olufemi Soneye.\nIt reads,” The production, which commenced on the 6th of May 2024, with 6,000 barrels of oil, is expected to be ramped up to 40,000 barrels per day by May 27th, 2024.\n“The first oil flow from OML 13 is a historic milestone in the partnership between NNPC E&P Ltd and NOSL, highlighting their dedication to driving growth and development in Nigeria’s oil and gas sector, which remains a vital component of the nation’s economy.\n“The achievement does not only signify the culmination of rigorous planning and execution by the teams involved but also represents

In [13]:
import openai

In [ ]:
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)



In [ ]:
# Prompt for generating topic tags
topic_prompt = f"Generate 5 topic tags for the given article:\n{article_text}\n---\n"
# Prompt for generating article category tags
category_prompt = f"Generate 5 article category tags for the given article:\n{article_text}\n---\n"

In [ ]:
t = loader.load()
t

[Document(page_content='By Chioma Onuegbu Uyo\nGOVERNOR Umo Eno has continued to demonstrate his readiness, determination, and commitment to fulfilling his contract with the people of Akwa Ibom State including creating job opportunities and enlarging the path for economic prosperity.\nAnd the governor at different fora emphasized that for his administration to achieve this contract there was need to revive the entrepreneurial spirit of the people through comprehensive training and support.\nAccordingly, as an ardent advocate of entrepreneurship development, as soon as he came on board, he hit the ground running, by putting in place variety of initiatives, popular among them is a Business School under the auspices of Ibom-Leadership Entrepreneurial Development (Ibom-LED).\nThe initiative which has its Headquarters at the E-Library Complex along IBB Avenue Uyo has Pastor ImaAbasi Jacob as the team lead and Mrs Ekaette Umoh as the Director General.\nAnd according to Ibom-LED team , the in

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install vllm

  Cloning https://github.com/casper-hansen/AutoAWQ_kernels.git (to revision main) to /tmp/pip-req-build-l2o1wjmp
  Running command git clone --filter=blob:none --quiet https://github.com/casper-hansen/AutoAWQ_kernels.git /tmp/pip-req-build-l2o1wjmp
  Resolved https://github.com/casper-hansen/AutoAWQ_kernels.git to commit 83d1f4b326a9067d0f94f089ef1bb47cf5377134
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl 

In [ ]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig
import awq_ext

In [ ]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

text = """<s>[INST] What is your favourite condiment? [/INST]
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>
[INST] Do you have mayonnaise recipes? [/INST]"""

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


In [ ]:
"Based on the provided news text content, extract 5 'topic tags' and extract 5 'article category tags' from the news text content"

In [ ]:
"""<s>[INST] Based on the provided news text content, extract 5 'topic tags' and extract 5 'article category tags' from the news text content"[/INST]
Topic Tags:Entrepreneurship Development,Economic Prosperity,Job Creation,Skills Training,Partnership Initiatives
Article Category Tags:Business & Economy,Government Initiatives,Youth Empowerment,Partnership Programs,Regional Development

In [ ]:
text = "By Chioma Onuegbu Uyo\nGOVERNOR Umo Eno has continued to demonstrate his readiness, determination, and commitment to fulfilling his contract with the people of Akwa Ibom State including creating job opportunities and enlarging the path for economic prosperity.\nAnd the governor at different fora emphasized that for his administration to achieve this contract there was need to revive the entrepreneurial spirit of the people through comprehensive training and support.\nAccordingly, as an ardent advocate of entrepreneurship development, as soon as he came on board, he hit the ground running, by putting in place variety of initiatives, popular among them is a Business School under the auspices of Ibom-Leadership Entrepreneurial Development (Ibom-LED).\nThe initiative which has its Headquarters at the E-Library Complex along IBB Avenue Uyo has Pastor ImaAbasi Jacob as the team lead and Mrs Ekaette Umoh as the Director General.\nAnd according to Ibom-LED team , the initiative was designed to provide skills to make Akwa Ibom economy private sector driven, to move Agriculture from subsistence to commercial, via comprehensive training programmes that covers all aspects of starting managing and scaling a business.\nAlso the programme was put in place to digitalize the state’s economy and to raise value-based leaders the development of the state.\nAnd so far the initiative which graduated its first- batch trainees of 400 in December 2023 could be said to be making progress given the number of Akwa Ibo young men and women who have been equipped with the requisite skills and knowledge and also supported with cash grants of N500,000 each to establish themselves and sustain their businesses.\nIt could be recalled that governor Umo Eno had started off the N500,000 cash grant support to Ibom-LED trainees when he announced the total sum of N250million for the business school in September 2023 at the Made in Akwa Ibom Trade Fair held at the Ibom-Leadership Entrepreneurial Development center.\nThe governor had explained that N50million was for the registration fee and training of the 400 participants, while five hundred thousand Naira (N500,000) should be disbursed to each trainee as a take-off grants after the completion of the three months Programme.\nJust last week, governor Umo Eno in his commitment to the success of the programme launched a N400million grant for disbursement to 800 beneficiaries of the Ibom-LED, Entrepreneurship Accelerator Programme (EAP).\nAdvising the 800 beneficiaries, Eno noted, “I would like to say clearly here, whether this Programme will continue or not depends on you because you are the Ambassadors of this Programme. You are not all totally new in business or entrepreneurship.\n“I am not concerned about the little SEED that will be put in your hands , but the training you have received. Everything you need to succeed is in your hands. For now don’t be politicians, just be entrepreneurs.\n” This is an opportunity for you to have a primary address. If you have a primary address politics is secondary, people will respect you for what you do. This little money is a SEED and if you plant it well, it will bring a mighty harvest.\n” I remember that in this state I opened my Hotel with just 5-rooms and few years later it had grown exponentially. The first thing is that you must make up your mind to succeed”\nIbom-LED has partnered other organizations within and outside the state such as the Partnership Initiative In the Niger Delta (PIND foundation) and Enterprise Development Center (EDC) based in Lagos state to promote and actualize its training programmes.\nAnd Ibom-LED is partnering the EDC on capacity building for 10,000 young Akwa Ibom who are 18 to 35years old.\nAccording to reports in December 2023 PIND foundation, a non-profit organization working with partners to build peace and equitable economic development in the Niger Delta partnered with Ibom-LED to implement the Youths Employment Pathway(YEP) project to expand its reach in Akwa Ibom state.\nThe skills programme on Technical Training and Leatherwork and Behavioural Skills and Entrepreneurship, aimed at transforming the mindset of the young trainees as well as enhance their enterprise acumen.\nSpeaking in the state during the signing of Memorandum of Understanding (MoU) recently between the the Entreprise Development Center and Ibom-LED, governor had expressed government readiness for more partnership to realize his vision in entrepreneurship development.\n“This partnership for me is critical. I thank the EDC for this partnership. And I ask for more of this support so as to realize the vision that we have set out and at the end of it Ibom-LED can truly say we have developed our Small and Medium Entreprises (SMEs)”, Eno had said.\nThe visible progress of Ibom-LED has started attracting commendations from individuals and groups such as the “Connect Initiative”\nA statement by Connect Initiative sighted online during the events marking governor Eno’s 60th birthday reads in part, ” Governor Eno has been in the forefront of efforts engendering business and economic growth with the creation of different initiative including the establishment of Ibom Leadership and Entrepreneurial Center aimed at cutting down poverty figures and providing opportunities for youths interested in making a career of their talents.\n“This this idea of not feeding them fish but teaching them how to fish is commendable”'

"Based on the provided news text content, extract 5 'topic tags' and extract 5 'article category tags' from the news text content"

In [ ]:
prompt = f"Generate 5 topic tags for the given article and 5 article category tags for the given article"
article = "By Chioma Onuegbu Uyo\nGOVERNOR Umo Eno has continued to demonstrate his readiness, determination, and commitment to fulfilling his contract with the people of Akwa Ibom State including creating job opportunities and enlarging the path for economic prosperity.\nAnd the governor at different fora emphasized that for his administration to achieve this contract there was need to revive the entrepreneurial spirit of the people through comprehensive training and support.\nAccordingly, as an ardent advocate of entrepreneurship development, as soon as he came on board, he hit the ground running, by putting in place variety of initiatives, popular among them is a Business School under the auspices of Ibom-Leadership Entrepreneurial Development (Ibom-LED).\nThe initiative which has its Headquarters at the E-Library Complex along IBB Avenue Uyo has Pastor ImaAbasi Jacob as the team lead and Mrs Ekaette Umoh as the Director General.\nAnd according to Ibom-LED team , the initiative was designed to provide skills to make Akwa Ibom economy private sector driven, to move Agriculture from subsistence to commercial, via comprehensive training programmes that covers all aspects of starting managing and scaling a business.\nAlso the programme was put in place to digitalize the state’s economy and to raise value-based leaders the development of the state.\nAnd so far the initiative which graduated its first- batch trainees of 400 in December 2023 could be said to be making progress given the number of Akwa Ibo young men and women who have been equipped with the requisite skills and knowledge and also supported with cash grants of N500,000 each to establish themselves and sustain their businesses.\nIt could be recalled that governor Umo Eno had started off the N500,000 cash grant support to Ibom-LED trainees when he announced the total sum of N250million for the business school in September 2023 at the Made in Akwa Ibom Trade Fair held at the Ibom-Leadership Entrepreneurial Development center.\nThe governor had explained that N50million was for the registration fee and training of the 400 participants, while five hundred thousand Naira (N500,000) should be disbursed to each trainee as a take-off grants after the completion of the three months Programme.\nJust last week, governor Umo Eno in his commitment to the success of the programme launched a N400million grant for disbursement to 800 beneficiaries of the Ibom-LED, Entrepreneurship Accelerator Programme (EAP).\nAdvising the 800 beneficiaries, Eno noted, “I would like to say clearly here, whether this Programme will continue or not depends on you because you are the Ambassadors of this Programme. You are not all totally new in business or entrepreneurship.\n“I am not concerned about the little SEED that will be put in your hands , but the training you have received. Everything you need to succeed is in your hands. For now don’t be politicians, just be entrepreneurs.\n” This is an opportunity for you to have a primary address. If you have a primary address politics is secondary, people will respect you for what you do. This little money is a SEED and if you plant it well, it will bring a mighty harvest.\n” I remember that in this state I opened my Hotel with just 5-rooms and few years later it had grown exponentially. The first thing is that you must make up your mind to succeed”\nIbom-LED has partnered other organizations within and outside the state such as the Partnership Initiative In the Niger Delta (PIND foundation) and Enterprise Development Center (EDC) based in Lagos state to promote and actualize its training programmes.\nAnd Ibom-LED is partnering the EDC on capacity building for 10,000 young Akwa Ibom who are 18 to 35years old.\nAccording to reports in December 2023 PIND foundation, a non-profit organization working with partners to build peace and equitable economic development in the Niger Delta partnered with Ibom-LED to implement the Youths Employment Pathway(YEP) project to expand its reach in Akwa Ibom state.\nThe skills programme on Technical Training and Leatherwork and Behavioural Skills and Entrepreneurship, aimed at transforming the mindset of the young trainees as well as enhance their enterprise acumen.\nSpeaking in the state during the signing of Memorandum of Understanding (MoU) recently between the the Entreprise Development Center and Ibom-LED, governor had expressed government readiness for more partnership to realize his vision in entrepreneurship development.\n“This partnership for me is critical. I thank the EDC for this partnership. And I ask for more of this support so as to realize the vision that we have set out and at the end of it Ibom-LED can truly say we have developed our Small and Medium Entreprises (SMEs)”, Eno had said.\nThe visible progress of Ibom-LED has started attracting commendations from individuals and groups such as the “Connect Initiative”\nA statement by Connect Initiative sighted online during the events marking governor Eno’s 60th birthday reads in part, ” Governor Eno has been in the forefront of efforts engendering business and economic growth with the creation of different initiative including the establishment of Ibom Leadership and Entrepreneurial Center aimed at cutting down poverty figures and providing opportunities for youths interested in making a career of their talents.\n“This this idea of not feeding them fish but teaching them how to fish is commendable”'

